<a href="https://colab.research.google.com/github/ThazSobral/tcc/blob/main/geographic_distance_tecnique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Técnica de distância geográfica.

by: João Porto de Albuquerque

## Ajuste

Primeiro fazemos todos os **ajustes** necessários para realizar o teste da técnica.

In [ ]:
# Montar do drive no notebook
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Em seguida fazemos a **importação** das primeiras **bibliotecas** e do nosso **dataset**.

In [ ]:
# Importar bibliotecas
import pandas as pd
import matplotlib.pyplot as plt

# Importar dataset
df = pd.read_csv('/content/drive/My Drive/TCC_Thalles Sobral/2-dados/germany/use-data/adjusted.csv', header=0)

# Exibir os dados
# df.head()

Podemos verificar **quantos tweets há** no nosso dataframe.

In [ ]:
# Mostrar quantos tweets há no nosso dataframe
print(f'Há { len(df) } tweets no total.')

Há 60524 tweets no total.


## Identificando regiões afetadas

1. As **características** dos **eventos** são definidos com base no **modelo digital de elevação** e os **dados oficiais** (ou seja, de sensores).
2. A identificação das **áreas de captação** foi feita utilizando um conjunto de ferrametas **ArcHydro** para ArcGIS. 
  * Primeiramente foram definidos os **canais de drenagem**, o que resultou em dados vetorias.
  * E por fim as **áreas de captação** foram identificadas utilizando as junções do rio, que foram definidas utilizando o arquivo vetorial de canal de drenagem.
3. O calculando o nível relativo da água (ou severidade da inundação) foi baseado na medição de 185 estações. Esse cálculo foi feito seguindo a definiçao a baixo:
~~~
relative_level = nível médio relativo
max_level = nível máximo de água diária
mean_level = nível médio de água da enchente (total)
~~~
`relative_level = max_level - mean_level`

> Em caso de **valores negativos** pode indicar que a estação **não foi afetada**, o que **valores positivos** podem indicar **o contrário**.

> Em caso de mais de uma estação na mesma bácia hidrográfica, o nível relativo de água é cálculado somando o valor relativo de cada estação e depois calculada a média aritmética do conjunto. Onde esse resultado definirá se a bácia foi afetada ou não.


## Coletando informações de mídia social (Twitter)

Essa etapa se dividi nas seguintes etapas:
* Filtragem baseada em palavras-chave;
* Análise do conteúdo, e;
* Condificação temática.

### Filtragem baseada em palavras-chave

> Antes filtrar o tweets por palavras-chave filtramos por campo de georreferência, ou seja, **somente tweets georreferenciados** passam para esse etapa. Isso pode ser feito na própria API do Twitter ou através de um script.

O filtro por palavras-chave considera palavras (alemão e inglês) relacionadas a eventos de inundação, como descrito a seguir:

**German**

* *hochwasser*
* *flut*
* *überschwemmung*

**Addiotional-words in German**

* *deich*
* *sandsack*

**English**

* *flood*


In [ ]:
# definir as palavras em inglês
key_words_english = ['flood']

# definir as palavras em alemão
key_words_german = ['hochwasser',
                     'flut',
                     'überschwemmung']

# definir as palavras adicionais em alemão
key_addiotional_words_german = ['deich',
                                'sandsack']

# combinar palavras-chave
key_words = key_words_english + key_words_german + key_addiotional_words_german

In [ ]:
# definir função para retornar somente aos tweets que contém as palavras-chaves
def select_tweets_with_key_words (df):
  return df[df.tweet.str.contains('|'.join(key_words), na=False)]

In [ ]:
# chamar a função para selecionar os tweets passando o dataframe completo
tweets_with_key_words = select_tweets_with_key_words(df)

# mostrar a quantidade de tweets que foram encontrados dentro do dataframe
print(f'Foram encontrados {len(tweets_with_key_words)} tweets contendo as palavras-chave definidas.')

# mostrar tweets que foram encotrados entro do dataframe
# tweets_with_key_words

Foram encontrados 405 tweets contendo as palavras-chave definidas.


### Análise de conteúdo

> **Só** são analisados os **tweets que contém** as **palavras-chave**.

Os tweets são **analisados e rotulados manualmente** (por três pesquisadores) dentro de **três classes**:
* (0) - fora do tópico;
* (1) - no tópico, mas não relevante, e;
* (2) - no tópico e relevante.

> O **tweet no tópico** (nesse trabalha) é aquele que se **refere ao evento** de inundação, mas **não contém informações relevantes**. 

> O **tweet relevante** (nesse trabalha) é definido como, aquele que **contém informações** que possam **contribuir para a consciência situacional**, ou seja, que pode ser útil para outras pessoas e/ou agências.

In [ ]:
# adicionar uma variável para cada rótulo da análise de conteúdo
topic_out = len(tweets_with_key_words.loc[(tweets_with_key_words['relevance'] == 0)])
topic_in = len(tweets_with_key_words.loc[(tweets_with_key_words['relevance'] == 1)])
topic_in_and_relevance = len(tweets_with_key_words.loc[(tweets_with_key_words['relevance'] == 2)])
# topic_out = len(df.loc[(df['relevance'] == 0)])
# topic_in = len(df.loc[(df['relevance'] == 1)])
# topic_in_and_relevance = len(df.loc[(df['relevance'] == 2)])

# exibir os rótulos com as respectivas quantias de tweets
print(f'''
 - fora do tópico: {topic_out};
 - no tópico: {topic_in}, e;
 - no tópico e relevante: {topic_in_and_relevance}
''')


 - fora do tópico: 35;
 - no tópico: 187, e;
 - no tópico e relevante: 183



In [ ]:
# exibir todos os tweets rotulados como no tópico
print(f'No geral são {topic_in + topic_in_and_relevance} tweets marcados como no tópico.')

No geral são 370 tweets marcados como no tópico.


### Codificação temática

Os tweets "no tópico", sendo relevantes ou não passaram por uma codificação que avalia seu conteúdo (também e forma manual). Essa avaliação consiste em **verificar o conteúdo e rotular o tweet** dentro das seguintes classe
* (0) - other;
* (1) - volunteer actions;
* (2) - media reports;
* (3) - traffic conditions;
* (4) - first-hand observations;
* (5) - official actions, e;
* (6) - infrastructure damage;

In [ ]:
# adicionar uma variável para cada rótulo da codificação temática
outhers = len(tweets_with_key_words[(tweets_with_key_words['theme']) == 0])
media_report = len(tweets_with_key_words[(tweets_with_key_words['theme']) == 1])
traffic_condition = len(tweets_with_key_words[(tweets_with_key_words['theme']) == 2])
firsthand_observation = len(tweets_with_key_words[(tweets_with_key_words['theme']) == 3])
official_action = len(tweets_with_key_words[(tweets_with_key_words['theme']) == 4])
damage_to_infrastruture = len(tweets_with_key_words[(tweets_with_key_words['theme']) == 5])
volunteer_action = len(tweets_with_key_words[(tweets_with_key_words['theme']) == 6])

# outhers = len(df[(df['theme']) == 0])
# media_report = len(df[(df['theme']) == 1])
# traffic_condition = len(df[(df['theme']) == 2])
# firsthand_observation = len(df[(df['theme']) == 3])
# official_action = len(df[(df['theme']) == 4])
# damage_to_infrastruture = len(df[(df['theme']) == 5])
# volunteer_action = len(df[(df['theme']) == 6])

# exibir a quantidade de tweets que são rotulados conforme a codificação temática
print(f'''
 - ação voluntaria: {volunteer_action}
 - relatório de mídia: {media_report}
 - condição de tráfego: {traffic_condition}
 - observação em primeira mão: {firsthand_observation}
 - ação oficial: {official_action}
 - dano a infraestrutura: {damage_to_infrastruture}
 - outros: {outhers}''')


 - ação voluntaria: 9
 - relatório de mídia: 71
 - condição de tráfego: 54
 - observação em primeira mão: 26
 - ação oficial: 69
 - dano a infraestrutura: 21
 - outros: 155


### Estabelecer relações geográficas entre tweets e os eventos

Para estabelecer a **relação de proximidade** para cada tweet é realizado um cálculo de resulta na **distância geográfica** (em metros) entre o **tweet e a bacia hidrográfica** mais próxima.

> tweets localizados **dentro da área** das bacias afetadas têm a **distância igual a 0m**.

> a relação de **gravidade** (severidade) é igual ao **nível relativo da água** da bacia que o tweet está localizado.

In [ ]:
# adicionando variável com quantidade e tweets entro das bacias
tweets_within_the_affected_basins = len(df[df['distance [m]'] == 0])

# exibir quantidade de tweets que estão dentro das bacias
print(f'São {tweets_within_the_affected_basins} tweets localizados dentro da bacia afetada')

São 1577 tweets localizados dentro da bacia afetada


## Analisando as relações geográficas entre as informações sobre os eventos e as mensagens da mídia social

Os objetivos dessa análise são dois:
* identificar **padrões espaciais** na ocorrência de tweets "no tópico" que podem estar **associados** à **distância** e o **nível relativo de água**, e;
* explorar mais as **possíveis diferenças** entre os **padrões de origem** e as **codificações dos tweets** no tópico.

> De acordo com o trabalho original foi constatado que há uma **maior probabilidade** de **tweets no tópico** ao redor das **bacias com níveis de água mais altos** comparados com bacias de níveis de água mais baixos.

> Para essa análise foram **utilizados** apenas tweets localizados **dentro de uma distância de 100 km** de bacias afetadas.

Nessa etapa é utilizado **GAM** (Generalized Aritmetic Model) com um **link logísitico** e **dois preditores** (**nível relativo de água** em metros e **log de base 10 da distância euclidiana** em quilometros para a **bacia mais próxima afetada**).

> Para **evitar resultados excessivamente influenciados** por valores extremos, foram feitos a seguitens alterações:
> * nível relativo da água entre + 1m e -1m;
> * distancias < 10km recebem 10km (antes de calcular o logaritmo).

O GAM implementado (do pacote R 'mgcv') **ajusta automaticamente de liberdade** efetivos do spline com um processo de validação cruzada generalizado, além de ser um 'bam' que é apropriado para **grandes conjuntos de dados**.

Foram ajustados **GAMs alternativos** que representem os preditores como termos **aditivos** (duas splines univariadas) ou **interativos** (uam spline bivariada). Para o modelo **aditivo** foi aplicado um **limite de liberdade** de **3 graus**, e para o modelo **interativo** um **limite** de **5 graus**, sendo a finalidade desses objetivos **evitar oscilações excessivas** nos splines resultantes.

Além dos GAMs serem utilizados por causa dos recursos visuais, mas também têm a função de **calcular** as **razões de chance e riscos relativos** (proporção de probabilidades) **associados** à **distância** e **nível relativo da água**. 

> O odds ratios (probablidade de relação) e risco relativo são calculados para **<= 10 km** versus **30 km** de distância das áreas afetadas, e para um nível relativo da água **+0,75 m** versus **-0,75 m** mantendo o **outro preditor constante**.
> Para esse cálculo é feito operando em cima de um preditor em diferentes níveis do outro preditor.
~~~
 - gravity_ratio = relação de gravidade
 - nível de água relativa da captura na localização do tweet
~~~
`gravity_ratio[tweet] = relative_level[tweet]`

> É aplicao o **bootstrap** de bloco espacial para obter **intervalos de confiança** percentil no nível de **95%**.
> O processo de **bootstrap** foi **repetido 2500 vezes** a fim de obter os **intervalos de confiança** de percentil de **95%**.

Exploração dos tweets para modelar a **probabilidade** de um **tweet no tópico pertencer a um subtipo** específico.

> Essa análise é realizada em tweets no tópico **dentro do buffer de 100 km** (resultando 320 no trabalho original).

> O modelo foi ajustado para **identificar padrões tweets como relevantes** (resultando 169 no trabalho original) **versus** os ***não relevantes** em relação a **distância** e ao **nível relativo da água**. Isso foi realizado para responder a **questão** dos **tweets mais relevantes** são fortemente **concetrado**s nas proximidades das **áreas afetadas** ou em **bacias** com **níveis relativos de água** mais **elevados**.

Cálculo da **relação geográfica** com o **tweet**
~~~
 - tweet_interface = relação do tweet
 - tweet_localization = localização do tweet
 - hydrographic_basin = bacia hidrográfica afetada
~~~
`tweet_interface = tweet_localization - hyrographic_basin`

> Como a amostra é pequena, somente **GAMs sem termo de interação** foram considerados, e apenas **resumos gráficos** e **numéricos básicos** são fornecidos para **análise exploratória** desses padrões.

In [ ]:
''' 
para mais informações:
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html
'''
# importe bibliotecas necessárias
import numpy as np
from sklearn.preprocessing import MaxAbsScaler

# extrai valores de severidade (níveis relativos de água) de cada dia
sev08 = df['sev08'].values
sev09 = df['sev09'].values
sev10 = df['sev10'].values

# concatene todos os valores de severidade coletados
sev = np.concatenate((sev08, sev09, sev10))

# instancia o modelo para normalização
model = MaxAbsScaler()
# treine o modelo para realizar normalização com esses valores
model.fit(sev.reshape(-1, 1))

# transforme os valores de acordo com o modelo
'''
convertendo os arrays em 2D (porque o modelo só funciona com 2D)
'''
sev08 = model.transform(sev08.reshape(-1,1))
# sev09 = model.transform(sev09.reshape(-1,1))
# sev10 = model.transform(sev10.reshape(-1,1))

# conveter os arrays para 1D novamente
sev08 = sev08.flatten()
sev09 = sev09.flatten()
sev10 = sev10.flatten()

In [ ]:
# atribui todas as diantancia que for menor ou igual a 10000 (10 km) atribui 10000
# df.loc[df['distance [m]'] <= 10000, 'distance'] = 10000

# passar a distancia pelo log10

In [ ]:
'''
para mais informações de correlação:
https://medium.com/brdata/correla%C3%A7%C3%A3o-direto-ao-ponto-9ec1d48735fb
'''
corr_dist_sev08 = df['distance [m]'].corr(df['sev08'])
corr_dist_sev09 = df['distance [m]'].corr(df['sev09'])
corr_dist_sev10 = df['distance [m]'].corr(df['sev10'])

print(f'Graus de correlação de distancia para os dias:\n08{corr_dist_sev08}\n09{corr_dist_sev09}\n10{corr_dist_sev10}')

Graus de correlação de distancia para os dias:
08-0.367923978782031
09-0.4470906959929109
10-0.46411051960810135


In [ ]:
# realizar bootstraping para separar os dados que seram passados para o modelo
import pandas
import numpy as np

# n = 5000
# values = np.random.uniform(size=(n, 5))

# columns = ['a', 'b', 'c', 'd', 'e']
# df = pandas.DataFrame(values, columns=columns)

# %timeit df.iloc[np.random.randint(n, size=n)]

# randlist = pandas.DataFrame(index=np.random.randint(n, size=n))
# %timeit df.merge(randlist, left_index=True, right_index=True, how='right')

# %timeit df.merge(pandas.DataFrame(index=np.random.randint(n, size=n)), left_index=True, right_index=True, how='right')

# df

# metodo mais rapido para bootstraping
# s = pd.Series(np.random.uniform(size=100))
# s

# TESTES

In [ ]:
# separar dados de treino e teste
# importe a biblioteca necessária
from sklearn.model_selection import train_test_split

# define as variaveis de entrada
X = df[['distance [m]', 'sev08']]
# define as variaveis de saida 
y = df['flood']

# separe os conjuntos de treino e teste
x_train, x_test, y_train, y_test = train_test_split(X, df.flood, test_size=.2)

In [ ]:
df.rename(columns={'distance [m]': 'distance'}, inplace=True)
df

,Unnamed: 0,hashtags,tweet,created_at,X,Y,distance,sev08,sev09,sev10,flood,relevance,theme
0,0,NaN,irgendjemand hat mir gestern gesagt das er mic...,2013-06-08 13:33:07+02,"429,79","5,915,818",122596,-24,-67,-127,0,0,0
1,1,NaN,"@ho1ger das gleiche bild übrigens im office, w...",2013-06-08 13:33:09+02,"708,059","5,342,364",66842,1,-79,-114,0,0,0
2,2,NaN,@sensatzionell bin selbst halber berliner. wen...,2013-06-08 13:33:10+02,"797,734","5,829,275",67270,-8,-10,-20,0,0,0
3,3,NaN,tired bro??!!! djeeerome @ ice 672 http://t.co...,2013-06-08 13:33:14+02,"522,889","5,881,956",33741,-24,-67,-127,0,0,0
4,4,NaN,und tschüss! (@ köln bonn airport (cgn) w/ 8 o...,2013-06-08 13:33:14+02,"367,861","5,638,052",71798,-54,-105,-161,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
60519,60519,gosh,why am i even still awake #gosh,2013-06-10 23:59:37+02,"512,133","5,280,630",410343,-157,-183,-199,0,0,0
60520,60520,NaN,@doyo_da toll. xd,2013-06-10 23:59:40+02,"355,787","5,709,831",305102,-49,-95,-151,0,0,0
60521,60521,NaN,family portrait! :) @ osteria tarantina http:/...,2013-06-10 23:59:44+02,"798,509","5,828,321",66394,-8,-10,-20,0,0,0
60522,60522,NaN,@j_taekwoon guten tag !!!! 즐거운하루하시구요! 저는이제 자러 ...,2013-06-10 23:59:50+02,"395,454","5,771,434",234868,-36,-59,-99,0,0,0


In [ ]:
# Aplicar GAM
# importe as bibliotecas necessárias
import statsmodels.api as sm
from statsmodels.gam.api import GLMGam, BSplines
# from statsmodels.gam.tests.test_penalized import df_autos

# definir os splines para o modelo dentro do dataset
# x_spline = df_autos[['weight', 'hp']]
x_spline = df[['distance', 'sev08']]

'''
instancie a classe, passando:
- os dados
- graus de liberdade
- grau do spline
'''
# bs= BSplines(x_spline, df=[12, 10], degree=[3,3])
bs= BSplines(x_spline, df=[3, 5], degree=[2, 2])

# definir penalização, onde o comprimento da lista deve ser igual ao numero de termos regulares no smoother
alpha = np.array([21833888.8, 6460.38479])
# alpha = np.array([cnsultar no trabalho])

'''
crie  o modelo a partir de uma fórmula e dataframe, passando:
- formula que especifica o modelo
- dados do modelo
- a instancia da classe
- a penalidade do modelo, onde o comprimento deve ser igual ao smoother
'''
# gam_bs = GLMGam.from_formula('city_mpg ~ fuel + drive', data=df_autos, smoother=bs, alpha=alpha)
gam_bs = GLMGam.from_formula('flood ~ distance + sev08', data=df, smoother=bs, alpha=alpha)

# estimar parãmetros e criar uma instancia da classe GLMGamResults
res_bs = gam_bs.fit()

# plotar as estimativas do modelo
# res_bs.plot_partial(0, cpr=True)
# res_bs.plot_partial(1, cpr=True)

# alpha = np.array([8283989284.5829611, 14628207.58927821])

# gam_bs = GLMGam.from_formula('city_mpg ~fuel + drive', data = df_autos, smoother=bs, alpha=alpha, family=sm.families.Poisson())

# res_bs = gam_bs.fit()

# res_bs.plot_partial(0, cpr=True)

# res_bs.plot_partial(1, cpr=True)

# gam_bs.select_penweight()[0]

# gam_bs.select_penweight_kfold()[0]

res_bs.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  flood   No. Observations:                60524
Model:                         GLMGam   Df Residuals:                 60517.00
Model Family:                Gaussian   Df Model:                         6.00
Link Function:               identity   Scale:                       0.0062392
Method:                         PIRLS   Log-Likelihood:                 67761.
Date:                Tue, 03 Nov 2020   Deviance:                       377.58
Time:                        14:52:26   Pearson chi2:                     378.
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0421      0.001     43.694      0.000       0.040       0.044
distance     2.213e-10   3.44e-09      0.064      0.949   -6.53e-09    6.97e-09
sev08        9.472e-05    6.3e-06     15.034      0.000    8.24e-05       0.000
distance_s0    -0.0099      0.003     -3.102      0.002      -0.016      -0.004
distance_s1     0.0049      0.002      3.102      0.002       0.002       0.008
sev08_s0       -0.0023      0.004     -0.575      0.566      -0.010       0.006
sev08_s1       -0.0381      0.001    -26.449      0.000      -0.041      -0.035
sev08_s2       -0.0116      0.003     -4.233      0.000      -0.017      -0.006
sev08_s3        0.0550      0.003     20.749      0.000       0.050       0.060
===============================================================================
"""

In [ ]:
# Teste PyGAM
!pip install pygam

In [ ]:
from pygam import GAM, s, f
from pygam import PoissonGAM

first_gam = GAM(s(0, n_splines=5) + s(1) + f(2) + s(3), distribution='gamma', link='log')
second_gam = PoissonGAM(s(0, n_splines=5) + s(1) + f(2) + s(3))

In [ ]:
# verificar os resultados